# API para Linha IMX

In [ ]:
def parse_aux(tuples):
    dict_file = {}
    for t in tuples:
        m = re.match(r'^#(.*)', t[0])
        if m:
            key = m[1].lower()
        else:
            key = t[0].lower()
        if re.search(r'path|file|start time|finish time', key):
            value = t[1]
        else:
            numbers = re.findall(r'-?\d+\.?\d*', t[1])
            if numbers == []:
                value = t[1]
            else:
                numbers = [float(s) for s in numbers]
            if len(numbers) == 1:
                value = numbers[0]
            elif len(numbers) > 1:
                value = {'x': numbers[0], 'y': numbers[1]}
        dict_file[key] = value
    return dict_file

In [ ]:
def parser_sample(path):
    with open(path) as file:
        file_string = file.read()
    list_s = re.split('\n', file_string)
    
    i = 0
    regex = re.compile(r'SAMPLE MOTORS')
    for l in list_s:
        if regex.search(l):
            idx_motor_s = i
            break
        i += 1
    i = 0
    for l in list_s[idx_motor_s:]:
        if l == '':
            idx_motor_f = i
            break
        i += 1

    list_motor = list_s[idx_motor_s:idx_motor_s + idx_motor_f]
    list_sample = list_s[0:idx_motor_s]
    list_sample.extend(list_s[idx_motor_s + idx_motor_f:])

    regex = re.compile(r'(.*) [=|-] (.*).*')
    tuples_sample = [(m.group(1), m.group(2)) for l in list_sample for m in [regex.search(l)] if m]
    tuples_motor = [(m.group(1), m.group(2)) for l in list_motor for m in [regex.search(l)] if m]
    
    dict_file = parse_aux(tuples_sample)
    dict_motor = parse_aux(tuples_motor)
    dict_file['motor positions'] = dict_motor
    
    return dict_file

In [ ]:
def parser_RAfT(path):
    with open(path) as file:
        file_string = file.read()
    list_s = re.split('\n', file_string)
    regex = re.compile(r'(.*) = (.*).*')
    tuples_sample = [(m.group(1), m.group(2)) for l in list_s for m in [regex.search(l)] if m]
    
    return parse_aux(tuples_sample)

In [ ]:
def add_proposal(proposal_id, collection, **kargs):
    document = {'proposta': proposal_id}
    document.update(kargs)
    collection.insert_one(document)

In [ ]:
def add_experiment(proposal_id, collection, **kwargs):
    collection.update_one({'proposta': proposal_id}, { '$push': { 'experimentos': kwargs } } )

In [ ]:
def choose_experiment(experiments):
    if len(experiments) > 1:
        n = 1
        print('There are more than one experiment in this proposal.') 
        for experiment in experiments:
            print('Experiment ', n, ',')
            for key, value in experiment.items():
                if key != 'amostras':
                    print('\t', key, ' : ', value)
            n += 1
        return str(int(input('Please choose one experiment ')) - 1)
    else:
        return '0'

In [ ]:
def choose_sample(samples):
    if len(samples) > 1:
        n = 1
        print('There are more than one sample in this experiment.') 
        for sample in samples:
            print('Sample ', n, ',')
            for key, value in sample.items():
                if key != 'medidas':
                    print('\t', key, ' : ', value)
            n += 1
        return str(int(input('Please choose one sample ')) - 1)
    else:
        return '0'

In [ ]:
def add_sample(proposal_id, collection, **kwargs):
    proposal = collection.find({'proposta': proposal_id})[0]
    idx = choose_experiment(proposal['experimentos'])
                    
    key = 'experimentos.' + idx + '.amostras'
    collection.update_one({'proposta': proposal_id}, { '$push': { key: kwargs } } )

In [ ]:
def add_measurements(proposal_id, collection, sample_path, RAfT_path):
    proposal = collection.find({'proposta': proposal_id})[0]
    idx_exp = choose_experiment(proposal['experimentos'])
    idx_sam = choose_sample(proposal['experimentos'][int(idx_exp)]['amostras'])
    
    args = parser_sample(sample_path)
    args['reconstructions'] = [parser_RAfT(RAfT_path)]
    
    key = 'experimentos.' + idx_exp + '.amostras.' + idx_sam + '.medidas'
    collection.update_one({'proposta': proposal_id}, { '$push': { key: args } } )

In [ ]:
def search_doc(doc, level_s, level_r, key, value):
    if level_s == 'experimento':
        if level_r == 'experimento':
            return [exp for exp in doc['experimentos'] if exp[key] == value]
        elif level_r == 'amostra':
            return [sample for exp in doc['experimentos'] if exp[key] == value for sample in exp['amostras']]
        elif level_r == 'medida':
            return [m for exp in doc['experimentos'] if exp[key] == value for sample in exp['amostras'] for m in sample['medidas']]
        
    elif level_s == 'amostra':
        if level_r == 'experimento':
            return [exp for exp in doc['experimentos']]
        elif level_r == 'amostra':
            return [sample for exp in doc['experimentos'] for sample in exp['amostras'] if sample[key] == value]
        elif level_r == 'medida':
            return [m for exp in doc['experimentos'] for sample in exp['amostras'] if sample[key] == value for m in sample['medidas']]
        
    elif level_s == 'medida':
        if level_r == 'experimento':
            return [exp for exp in doc['experimentos']]
        elif level_r == 'amostra':
            return [sample for exp in doc['experimentos'] for sample in exp['amostras']]
        elif level_r == 'medida':
            return [m for exp in doc['experimentos'] for sample in exp['amostras'] for m in sample['medidas'] if m[key] == value]